In [123]:
import sys
# sys.path.append("/Users/bubble/Desktop/Project/Infrasound Sensor/Layout/Code/acousticsensor")
import numpy as np
import gdsfactory as gf

cell_temp = gf.Component()

In [124]:
# pad and lines
pad = gf.Component()

# lines
line_base = gf.Component()
line_half = gf.Component()
# define the cross section
w1 = 10
w2 = 1  
s1 = gf.Section(width=w1, offset=0, layer=(2, 0), name="1", port_names=("o1", "o2"))
X1 = gf.CrossSection(sections=[s1])
s2 = gf.Section(width=w2, offset=0, layer=(2, 0), name="1", port_names=("o1", "o2"))
X2 = gf.CrossSection(sections=[s2])
# define the path
# define distance
d1 = 200
radius = 20
h1 = 20
d5 = 1.5
p1 = gf.path.straight(length=d1)
p = p1
wg1 = gf.path.extrude(p, X1)
p4 = gf.path.arc(radius=radius, angle=-90)
Xtrans = gf.path.transition(cross_section1=X1, cross_section2=X2, width_type="sine")
curve_transition = gf.path.extrude_transition(p4, Xtrans)

wg1_ref = line_base << wg1
wg2_ref = line_base << curve_transition
wg2_ref.connect("o1", wg1_ref.ports["o2"])

P = gf.Path()
p5 = gf.path.straight(length=h1)
p6 = gf.path.straight(length=d5)
P += p5
P.end_angle += 90  # "Turn" 90 deg (left)
P += p6
wg3 = gf.path.extrude(P, X2)
wg3_ref = line_base << wg3
wg3_ref.connect("o1", wg2_ref.ports["o2"])

# line_base.show()
d = d1+radius+d5
line_base_ref1 = line_half << line_base
line_base_ref2 = line_half << line_base
line_base_ref2.dmirror_x(d)
# add connection
w_con = (d5-w2/2)*2
h_con = 3-w2
connection = gf.components.rectangle(size=(w_con, h_con), layer=(2, 0))
line_half.add_ref(connection).move((-w_con/2+3*d, radius+h1-h_con-w2/2))
pad.add_ref(line_half)
pad.add_ref(line_half).drotate(angle=180, center=(2*d, 0))
# pad.show()




Unnamed_549: ports [], vinsts=[] info=Info() kcl=KCLayout(name='DEFAULT', layout=<klayout.dbcore.Layout object at 0x1313293d0>, layer_enclosures=LayerEnclosureModel(root={'5c226817': LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosure'), '4fb2b992': LayerEnclosure(layer_sections={}, main_layer=9/0, yaml_tag='!Enclosure'), '56396fc8': LayerEnclosure(layer_sections={}, main_layer=SLAB90 (3/0), yaml_tag='!Enclosure'), '6b4bcfc2': LayerEnclosure(layer_sections={}, main_layer=N (20/0), yaml_tag='!Enclosure'), 'a5f0d270': LayerEnclosure(layer_sections={}, main_layer=P (21/0), yaml_tag='!Enclosure')}), cross_sections={'5c226817_10000': SymmetricalCrossSection(width=10000, enclosure=LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosure'), name='5c226817_10000'), '5c226817_400': SymmetricalCrossSection(width=400, enclosure=LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosure'), name='5c226817_400'), '5c226817_10

In [125]:
# pads
pad_size = 500
h2 = 100
w3 = 20
line_vertical = gf.components.rectangle(size=(w3, h2), layer=(2, 0))
pad.add_ref(line_vertical).move((2*d-w3/2, -h2-w1/2))
sg_pad = gf.components.rectangle(size=(pad_size, pad_size), layer=(2, 0))
pad.add_ref(sg_pad).move((-pad_size, w1/2-pad_size))
pad.add_ref(sg_pad).move((4*d, w1/2-pad_size))
pad.add_ref(sg_pad).move((2*d-pad_size/2, -w1/2-pad_size-h2))
# pad.show()



Unnamed_549: ports ['e1', 'e2', 'e3', 'e4'], vinsts=[] info=Info() kcl=KCLayout(name='DEFAULT', layout=<klayout.dbcore.Layout object at 0x1313293d0>, layer_enclosures=LayerEnclosureModel(root={'5c226817': LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosure'), '4fb2b992': LayerEnclosure(layer_sections={}, main_layer=9/0, yaml_tag='!Enclosure'), '56396fc8': LayerEnclosure(layer_sections={}, main_layer=SLAB90 (3/0), yaml_tag='!Enclosure'), '6b4bcfc2': LayerEnclosure(layer_sections={}, main_layer=N (20/0), yaml_tag='!Enclosure'), 'a5f0d270': LayerEnclosure(layer_sections={}, main_layer=P (21/0), yaml_tag='!Enclosure')}), cross_sections={'5c226817_10000': SymmetricalCrossSection(width=10000, enclosure=LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosure'), name='5c226817_10000'), '5c226817_400': SymmetricalCrossSection(width=400, enclosure=LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosure'), name='5c2268

In [126]:
# cell
L = [250, 500, 750, 1000]
w = 5
block = gf.Component()
block_temp = gf.Component()
origin = [0, 0]

for i in range(len(L)):
    # add beam
    beam = gf.components.rectangle(size=(w, L[i]), layer=(9, 0))
    # length mark
    T = gf.components.text(f"{L[i]}", size=50, layer=(1, 0))
    T_ref = block_temp << T
    # add pad
    pad_ref = block_temp << pad
    # add backside frame
    frame_temp = gf.Component()
    size = 1250
    frame = gf.components.rectangle(size=(size, size), layer=(3, 0))
    frame_temp.add_ref(frame).movex(-size/2+w/2)
    beam_temp = gf.boolean(A = frame_temp, B = beam,  operation="not", layer1=(3, 0), layer2=(9, 0), layer=(1, 0))
    
    # add backside etch
    size2 = 371.72 + size
    backside_temp = gf.Component()
    backside = gf.components.rectangle(size=(size2, size2), layer=(3, 0))
    backside_temp.add_ref(backside).move((-(size2-size)/2-size/2+w/2, -(size2-size)/2))
    beam_temp << backside_temp
    beam_ref = block_temp << beam_temp
    # adjust position
    spacing = 2500
    beam_ref.movey(spacing*i-250)
    T_ref.movey(spacing*i-250-700)
    pad_ref.move((-3*d+w/2, spacing*i-250-radius))

block << block_temp
# block.show()

Unnamed_555: ports [], vinsts=[] info=Info() kcl=KCLayout(name='DEFAULT', layout=<klayout.dbcore.Layout object at 0x1313293d0>, layer_enclosures=LayerEnclosureModel(root={'5c226817': LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosure'), '4fb2b992': LayerEnclosure(layer_sections={}, main_layer=9/0, yaml_tag='!Enclosure'), '56396fc8': LayerEnclosure(layer_sections={}, main_layer=SLAB90 (3/0), yaml_tag='!Enclosure'), '6b4bcfc2': LayerEnclosure(layer_sections={}, main_layer=N (20/0), yaml_tag='!Enclosure'), 'a5f0d270': LayerEnclosure(layer_sections={}, main_layer=P (21/0), yaml_tag='!Enclosure')}), cross_sections={'5c226817_10000': SymmetricalCrossSection(width=10000, enclosure=LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosure'), name='5c226817_10000'), '5c226817_400': SymmetricalCrossSection(width=400, enclosure=LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosure'), name='5c226817_400'), '5c226817_10

In [127]:
# structure for each die
# structure for a block
fblock = gf.Component()
for i in range(6):
    block_ref = fblock << block
    block_ref.move((2500*(i+1), 0))


In [ ]:
# boolean operation

cell_temp.add_ref(fblock).move((-1000, 1600))

# frame
frame1 = gf.components.rectangle(size=(15000, 10830), layer=(20, 0))
cell_temp << frame1
frame2 = gf.components.rectangle(size=(20000, 15830), layer=(21, 0))
frame2_ref = cell_temp << frame2
frame2_ref.move((-2500, -2500))

cell_cantilever_resistive = gf.Component()
cell_ref = cell_cantilever_resistive << cell_temp
cell_ref.move((2500, 12500))
# cell_cantilever_resistive.show()
# cell_cantilever_resistive.write_gds("mesh.gds")
# cell_cantilever_resistive.plot()